# Random Projection simulation based on Johnson Lindenstrauss Lemma

Given $\epsilon, \beta > 0$ let $k_0 = \frac{4+2\beta}{\epsilon^2/2 - \epsilon^3/3}\log{n}$. For every integer $k\ge k_0$, let $R$ be a $d\times k$ random matrix with $R(i,j) = r_{ij}$ where each $r_{ij}$ are independent random variables with pmf:

$r_{ij} = \sqrt{3}\times\begin{cases}
+1 & \text{w/ prob } 1/6\\
0  & \text{w/ prob } 2/3\\
-1 & \text{w/ prob } 1/6
\end{cases}$

Let $f: \mathbb{R}^{d} \rightarrow \mathbb{R}^{k} = \frac{1}{\sqrt{k}}R$

With probability at least $1-n^{-\beta}$, for all $u,v\in P$
$$(1-\epsilon)||u-v||_2^2 \le ||\frac{1}{\sqrt{k}}Ru-\frac{1}{\sqrt{k}}Rv||_2^2 \le (1+\epsilon)||u-v||_2^2$$

Work credit: http://people.ee.duke.edu/~lcarin/p93.pdf

In [4]:
import numpy as np
from tqdm import tqdm
import math


In [1]:
def create_matrix (k, d):
    result = np.ones((k, d))
    for i in tqdm(range(len(result)), desc="populating matrix"):
        for j in range(len(result[i])):
            seed = np.random.randint(1, 7)
            if seed == 1:
                result[i, j] = 3**0.5
            elif seed == 6:
                result[i, j] = (-1)*(3**0.5)
            else:
                result[i, j] = 0
    return result

def valudiate (m):
    neg_sqrt_3 = 0
    sqrt_3 = 0
    zero = 0
    for i in range(np.shape(m)[0]):
        for j in range(np.shape(m)[1]):
            if m[i, j] == 3**0.5:
                sqrt_3 += 1
            elif m[i, j] == (-1)*3**0.5:
                neg_sqrt_3 += 1
            elif m[i, j] == 0:
                zero += 1
            else:
                print("error")
    return [sqrt_3, neg_sqrt_3, zero]

In [5]:
def sparse_jl(a, e, b, k):
    n, d = np.shape(a)
    k0 = (4 + 2 * b)/(((e**2) / 2) - ((e**3) / 3)) * math.log(n)
    if k < k0:
        print("k0 is " + str(k0))
        return []
    else:
        result = np.ones((k, d))
        for i in range(len(result)):
            for j in range(len(result[i])):
                seed = np.random.randint(1, 7)
                if seed == 1:
                    result[i, j] = 3**0.5
                elif seed == 6:
                    result[i, j] = (-1)*(3**0.5)
                else:
                    result[i, j] = 0
    return np.matmul(a, np.transpose(result))


# Las-Vegas Version of the above random projection

In [6]:
def validate(matrix, epsilon, row):
    uv_combo = [[a, b] for a in range(row) for b in range(row) if a != b]
    for i in uv_combo:
        u = matrix[i[0]]
        v = matrix[i[1]]
        lower_bound = (1 - epsilon) * np.linalg.norm(np.subtract(u, v))**2
        upper_bound = (1 + epsilon) * np.linalg.norm(np.subtract(u, v))**2
        verify = np.linalg.norm(
            ((1/(row**0.5)) * np.matmul(matrix, u) - (1/(row**0.5)) * np.matmul(matrix, v)))
        #print("lower bound: " + str(lower_bound) + " value is: " + str(verify) + " upper bound: " + str(upper_bound))
        if lower_bound > verify or upper_bound < verify:
            return False
    return True


In [7]:
def las_vegas_jl(a, e, b, k):
    succeeded = False
    final_result = []
    count = 0
    while succeeded == False:
        result = sparse_jl(a, e, b, k)
        count += 1
        succeeded = validate(result, e, np.shape(result)[0])
        final_result = result
    return (final_result, count)
